In [2]:
import os
import sys
os.environ["PYSPARK_SUBMIT_ARGS"]='--packages com.databricks:spark-csv_2.10:1.2.0 pyspark-shell'
spark_home = os.environ.get('SPARK_HOME', None)
sys.path.insert(0, spark_home + "/python")
execfile(os.path.join(spark_home, 'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.1.1
      /_/

Using Python version 2.7.6 (default, Oct 26 2016 20:30:19)
SparkSession available as 'spark'.


In [3]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import *


In [4]:
users = sc.textFile("ml-100k/u.data")

In [5]:
users = sqlContext.createDataFrame(users.map(lambda x: x.split("\t")),['user_id','item_id','rating','timestamp'])
users.show()

+-------+-------+------+---------+
|user_id|item_id|rating|timestamp|
+-------+-------+------+---------+
|    196|    242|     3|881250949|
|    186|    302|     3|891717742|
|     22|    377|     1|878887116|
|    244|     51|     2|880606923|
|    166|    346|     1|886397596|
|    298|    474|     4|884182806|
|    115|    265|     2|881171488|
|    253|    465|     5|891628467|
|    305|    451|     3|886324817|
|      6|     86|     3|883603013|
|     62|    257|     2|879372434|
|    286|   1014|     5|879781125|
|    200|    222|     5|876042340|
|    210|     40|     3|891035994|
|    224|     29|     3|888104457|
|    303|    785|     3|879485318|
|    122|    387|     5|879270459|
|    194|    274|     2|879539794|
|    291|   1042|     4|874834944|
|    234|   1184|     2|892079237|
+-------+-------+------+---------+
only showing top 20 rows



Глобальное среднее 𝞵 (average_rating) по всему датасету. Сумма всех оценок по всем фильмам / Количество всех оценок по всем фильмам.



In [6]:
#global M
Mr =  users.agg(F.mean('rating').alias("Mr")).rdd.flatMap(lambda x: x).collect()
Mr = Mr[0]
Mr

3.52986

Базовый предиктор для каждого пользователя (суммирование по фильмам, оцененным данным пользователем). Здесь Ia — множество фильмов, по которым у пользователя есть рейтинги, а |Ia| — их количество.



In [7]:
#Предиктор Bu для пользователя
users_bu = users.withColumn(('r-M'), (col('rating')-Mr)).\
        groupby('user_id').agg(F.sum('r-M').alias('r-M'), F.count('item_id').alias('N')).\
        select('user_id',((1 / (10 + col('N')))*col('r-M')).alias('bu'))

In [8]:
users_bu.show()

+-------+--------------------+
|user_id|                  bu|
+-------+--------------------+
|    296|  0.6057998726114655|
|    467|  0.1238177777777776|
|    691|  0.5248685714285714|
|    675| 0.13601727272727257|
|    829|0.014715675675675487|
|    125|-0.08559645833333303|
|    451| -0.7215396296296293|
|    800| 0.16220842105263142|
|    853|-0.44557372549019575|
|    666| 0.13013450980392194|
|    870|-0.07359261648745483|
|    919|-0.05717894273127723|
|    926|-0.15324000000000015|
|      7|   0.424858159806296|
|    124|-0.02107764705882...|
|     51| 0.02464303030303015|
|    447| 0.06274805369127517|
|    591| 0.11161446808510626|
|    307| 0.24307934426229535|
|    475| 0.04675999999999985|
+-------+--------------------+
only showing top 20 rows



In [441]:
#джойн с общей таблицей пользователей, чтобы посчитать следующий предиктор
films = users.join(users_bu, users.user_id == users_bu.user_id, "inner").drop(users.user_id).drop("timestamp")
films.show(100)

+-------+------+-------+------------------+
|item_id|rating|user_id|                bu|
+-------+------+-------+------------------+
|    705|     5|    296|0.6057998726114655|
|    508|     5|    296|0.6057998726114655|
|     20|     5|    296|0.6057998726114655|
|    228|     4|    296|0.6057998726114655|
|    222|     5|    296|0.6057998726114655|
|    429|     5|    296|0.6057998726114655|
|    855|     5|    296|0.6057998726114655|
|    248|     5|    296|0.6057998726114655|
|    258|     5|    296|0.6057998726114655|
|    242|     4|    296|0.6057998726114655|
|     48|     5|    296|0.6057998726114655|
|    286|     5|    296|0.6057998726114655|
|    272|     5|    296|0.6057998726114655|
|    510|     5|    296|0.6057998726114655|
|    275|     4|    296|0.6057998726114655|
|    427|     5|    296|0.6057998726114655|
|     83|     5|    296|0.6057998726114655|
|    961|     5|    296|0.6057998726114655|
|    544|     4|    296|0.6057998726114655|
|     32|     4|    296|0.605799

Базовый предиктор для каждого фильма (суммирование по пользователям, поставившим оценку данному фильму). Здесь Ui — множество пользователей, которые оценили данный фильм, а |Ui| — их количество.


In [443]:
# расчет bf - предиктора для фильмов
films_bu = films.withColumn(('r-M-b'), (col('rating')-Mr-col('bu'))).\
                    groupby('item_id').agg(F.sum('r-M-b').alias('r-M-b'), F.count('user_id').alias('N')).\
        select('item_id',((1 / (25 + col('N')))*col('r-M-b')).alias('bf'))

In [11]:
films_bu.show()

+-------+--------------------+
|item_id|                  bf|
+-------+--------------------+
|    467| 0.16690701840272257|
|   1090| -0.5930556738185065|
|    675| 0.04475927935836765|
|    829|-0.42705134001999234|
|   1159|0.006318234718819636|
|    691| 0.05546248651815933|
|    296|0.015819748575955868|
|   1436|-0.04295878652633444|
|   1512| 0.10401325746334374|
|   1572|-0.03296769642673...|
|    125|0.014627800938313704|
|    451|-0.16143428135568216|
|   1394|-0.02895808441574...|
|    800|-0.26919053472157284|
|    944|-0.29616250278667783|
|    853| 0.16828496950765312|
|   1669|-0.03151598855689...|
|   1372|-0.00766928999193...|
|   1265| -0.1338290522533187|
|    919| 0.20844480341617108|
+-------+--------------------+
only showing top 20 rows



In [12]:
#сводим все в общую табличку для расчета общего предиктора
users_p = films.join(films_bu, films.item_id == films_bu.item_id, "inner").drop(films_bu.item_id)

In [13]:
users_p.show(100)

+------+-------+--------------------+-------+--------------------+
|rating|user_id|                  bu|item_id|                  bf|
+------+-------+--------------------+-------+--------------------+
|     2|    682| -0.3824306601466988|   1090| -0.5930556738185064|
|     3|    690|-0.35147119999999965|   1090| -0.5930556738185064|
|     4|    504| 0.08814229885057472|   1090| -0.5930556738185064|
|     4|    693| -0.3695026829268289|   1090| -0.5930556738185064|
|     2|    183| -0.2870250793650794|   1090| -0.5930556738185064|
|     3|     92| -0.2778534673366829|   1090| -0.5930556738185064|
|     2|    145|-0.18845325153374193|   1090| -0.5930556738185064|
|     1|    437| 0.04998835164835185|   1090| -0.5930556738185064|
|     2|    346|-0.22789645320197005|   1090| -0.5930556738185064|
|     2|    757|-0.15316340909090875|   1090| -0.5930556738185064|
|     3|    417|-0.27573039999999954|   1090| -0.5930556738185064|
|     2|    870|-0.07359261648745483|   1090| -0.5930556738185

In [312]:
#Расчет юзер предиктора для 234-ого пользователя
bu234 = users_p.filter(users_p.user_id == 234).select('bu').dropDuplicates().\
        rdd.flatMap(lambda x: x).collect()[0]

In [313]:
bu234

-0.39863836734693886

In [14]:
users_p.count()

100000

Базовый предиктор для каждого пользователя и каждого фильма:


In [237]:
#Расчитываем базовый общий предиктор bu+M+bf и отнимаем его ото всех рейтингов, нормируя их
users_b = users_p.withColumn('bu+M+bf',(col('bf')+col('bu')+Mr)).withColumn('r-b', (col('rating') - col('bu+M+bf')))
users_b =users_b.select('item_id','user_id','r-b','bu+M+bf','bu')
users_b.show(100)

+-------+-------+--------------------+------------------+--------------------+
|item_id|user_id|                 r-b|           bu+M+bf|                  bu|
+-------+-------+--------------------+------------------+--------------------+
|   1090|    870| -0.8632117096940388| 2.863211709694039|-0.07359261648745483|
|   1090|    586| 0.25613181018214215| 2.743868189817858|-0.19293613636363607|
|   1090|    276| -1.8734189473936151| 2.873418947393615|-0.06338537878787867|
|   1090|    221|-0.01783278771995...|3.0178327877199553| 0.08102846153846156|
|   1090|    320|-0.11608213105954279| 3.116082131059543| 0.17927780487804892|
|   1090|    268|-0.36640763979096036|2.3664076397909604| -0.5703966863905334|
|   1090|    303| -1.7759698322543689| 2.775969832254369|-0.16083449392712496|
|   1090|    437| -1.9867926778298455|2.9867926778298455| 0.04998835164835185|
|   1090|    399| -0.3622592197985157|2.3622592197985157| -0.5745451063829782|
|   1090|    682| -0.5543736660347949| 2.55437366603

In [16]:
films = users_b.groupby('item_id').count()
films.count()

1682

Найдите попарные меры близости (косинус) для всех фильмов, используя очищенные оценки из пункта 1, часть 4. Суммирование идет по всем пользователям.


In [24]:
#Функция преобразует переданные значения колонок в строку, разделенную ':'
from pyspark.sql.types import ArrayType, StringType


def OneV(v):
    s = str(v[0])+':'+str(v[1])
    return s



In [25]:
# операция возвращает в каждую строку датасета с фильмом  строчку с номером юзера : нормализованным рейтингом
#По сути она символизирует собой вектор оценки фильма конкетным юзером.

from pyspark.sql.functions import udf, array

OneV=udf(OneV, StringType())

users_b=users_b.withColumn("SV", OneV(array('user_id','r-b')))


In [26]:
users_b.show()

+-------+-------+--------------------+------------------+--------------------+
|item_id|user_id|                 r-b|           bu+M+bf|                  SV|
+-------+-------+--------------------+------------------+--------------------+
|   1090|    870| -0.8632117096940393|2.8632117096940393|870:-0.8632117096...|
|   1090|    586| 0.25613181018214215| 2.743868189817858|586:0.25613181018...|
|   1090|    796|  0.9944181738185058| 3.005581826181494|796:0.99441817381...|
|   1090|    267| -0.3212961210532894|3.3212961210532894|267:-0.3212961210...|
|   1090|    221|-0.01783278771995...|3.0178327877199553|221:-0.0178327877...|
|   1090|    320|-0.11608213105954279| 3.116082131059543|320:-0.1160821310...|
|   1090|     95| -1.8281200206259385|2.8281200206259385|95:-1.82812002062...|
|   1090|    741|  -1.874475905128862| 2.874475905128862|741:-1.8744759051...|
|   1090|    268|-0.36640763979096036|2.3664076397909604|268:-0.3664076397...|
|   1090|    405|-0.26410443327654676|1.264104433276

In [ ]:
#соберем все записи оценок  фильма  юзером в лист. Для этого агрегируем датафрейм по фильмам, и 
#в каждой строчке этого листа будут проставлены номера юзеров, оценивших фильм, и их рейтинги                                                                                                                                                                                                                ол иопорарпавпавд
users_b1 =users_b.groupby('item_id').agg(collect_list(users_b.SV).alias('List'))

In [28]:
users_b1.show()

+-------+--------------------+
|item_id|                List|
+-------+--------------------+
|   1090|[586:0.2561318101...|
|   1159|[747:-2.043607343...|
|   1436|[207:-0.199952046...|
|   1512|[305:-0.519093429...|
|   1572|[405:-0.824192410...|
|    296|[782:0.1647645489...|
|    467|[747:-0.204196127...|
|    675|[334:0.4354197293...|
|    691|[863:-0.211860777...|
|    829|[577:-0.362335528...|
|    125|[7:0.030654039255...|
|   1372|[449:0.2865092899...|
|   1394|[782:1.2095423819...|
|   1669|[782:-0.787899713...|
|    451|[591:-0.480040186...|
|    800|[843:1.4967671702...|
|    853|[308:1.0791386177...|
|    944|[712:0.5444264562...|
|   1241|[354:0.3998460340...|
|   1265|[455:-0.370184013...|
+-------+--------------------+
only showing top 20 rows



In [54]:
#функция, формирующая  запись sparse вектора из переданного ей листа с данными. Можно было возвращать не запись 
# вектора, а сам вектор, но исторически так сложилось, что я продолжила именно этот вариант кода. 944 - это где-то 
# возможно стертое уже, посчитанное количество юзеров
from pyspark.mllib.linalg import VectorUDT, DoubleType

def SpV(l):
    import collections
    vec = {}
    for data in l:
        vec[(int(data.split(':')[0]))] = float(data.split(':')[1])
    od = collections.OrderedDict(sorted(vec.items()))
    t = (944, od.keys(), od.values())
    return str(t)


In [45]:
#просто проверила как работает Ordered dict
import collections

d = {2:3, 1:89, 4:5, 3:0}

od = collections.OrderedDict(sorted(d.items()))

od


OrderedDict([(1, 89), (2, 3), (3, 0), (4, 5)])

In [46]:
od.keys()

[1, 2, 3, 4]

In [47]:
od.values()

[89, 3, 0, 5]

In [ ]:
#для каждого фильма сформировали sparse vector, по где  место нумеруется юзером,а значение - проставленным рейтингом
# (или нулем, если не оценивал)
from pyspark.sql.types import ArrayType, StringType

users_b1 = users_b1.withColumn("V", udf(SpV, StringType())(col("List")))


In [56]:
users_b1.show()

+-------+--------------------+--------------------+
|item_id|                List|                   V|
+-------+--------------------+--------------------+
|   1090|[870:-0.863211709...|(944, [56, 92, 95...|
|   1159|[543:1.4608382870...|(944, [76, 183, 3...|
|   1436|[207:-0.199952046...|(944, [207, 655],...|
|   1512|[363:-2.173479301...|(944, [305, 363, ...|
|   1572|[405:-0.824192410...|(944, [405], [-0....|
|    296|[733:-1.086747953...|(944, [2, 139, 65...|
|    467|[666:0.1730984717...|(944, [6, 10, 13,...|
|    675|[118:0.4330357823...|(944, [7, 13, 21,...|
|    691|[863:-0.211860777...|(944, [9, 13, 100...|
|    829|[711:-1.418397144...|(944, [13, 54, 94...|
|    125|[800:-0.706696221...|(944, [1, 6, 7, 1...|
|   1372|[449:0.2865092899...|(944, [181, 189, ...|
|   1394|[451:-1.779362285...|(944, [181, 206, ...|
|   1669|[782:-0.787899713...|(944, [782], [-0....|
|    451|[125:0.7171707396...|(944, [5, 7, 11, ...|
|    800|[843:1.4967671702...|(944, [13, 21, 28...|
|    853|[27

In [316]:
# выгрузим все вектора во внешний лист. Они легкие в такой записи, но, возможно, это самая неоптимальная часть.
from pyspark.sql.types import FloatType

M = users_b1.sort(users_b1.item_id.cast(FloatType())).select('V').rdd.flatMap(lambda x: x).map(lambda x: (x)).collect()

In [317]:
M

[u'(944, [1, 2, 5, 6, 10, 13, 15, 16, 17, 18, 20, 21, 23, 25, 26, 38, 41, 42, 43, 44, 45, 49, 54, 56, 57, 58, 59, 62, 63, 64, 65, 66, 67, 70, 72, 73, 75, 77, 79, 81, 82, 83, 84, 89, 92, 93, 94, 95, 96, 97, 99, 101, 102, 106, 108, 109, 117, 120, 121, 124, 125, 128, 130, 131, 134, 137, 138, 141, 144, 145, 148, 150, 151, 157, 158, 160, 162, 168, 174, 177, 178, 181, 182, 184, 189, 193, 194, 198, 199, 200, 201, 202, 203, 204, 209, 210, 213, 216, 222, 223, 230, 231, 232, 234, 235, 242, 243, 244, 246, 247, 248, 249, 250, 251, 252, 253, 254, 256, 262, 263, 265, 268, 271, 274, 275, 276, 277, 279, 280, 286, 287, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 301, 303, 305, 307, 308, 311, 312, 313, 314, 320, 322, 324, 325, 326, 327, 330, 331, 332, 336, 338, 339, 340, 343, 344, 345, 347, 348, 350, 357, 359, 360, 363, 365, 371, 374, 378, 379, 380, 381, 387, 388, 389, 390, 393, 394, 395, 396, 398, 399, 401, 402, 403, 406, 407, 411, 412, 416, 417, 419, 422, 424, 425, 429, 432, 434, 435, 438, 

In [60]:
#выгрузим все номера фильмов
film = users_b.select('item_id').dropDuplicates().rdd.flatMap(lambda x: x).map(lambda x: int(x)).collect()

In [61]:
#выгрузим все номера юзеров
user =users_b.select('user_id').dropDuplicates().rdd.flatMap(lambda x: x).map(lambda x: int(x)).collect()


In [ ]:
# забродкастим М
M_bc = sc.broadcast(M)

In [ ]:
#забродскатим номера фильмов (хотя они, конечно, идут по порядку, и можно было  обойтись обычным range)
film_bc = sc.broadcast(sorted(film))

In [62]:
len(user)

943

In [279]:
#просто функция, которой можно передавать номера фильмов, а она посчитает косинусную близость между ними с помощью
# забродкастенной переменной М. поскольку М это список, и он начинается с нуля, от номеров фильмов отнимается 1
def calc_cosine_similarity(v1,v2):
    import math
    from pyspark.mllib.linalg import Vectors, VectorUDT
    a = Vectors.parse(M_bc.value[int(v1)-1]).asML()
    b =Vectors.parse(M_bc.value[int(v2)-1]).asML()
    cs = float(a.dot(b)) / (float(math.sqrt(a.dot(a))) *float(math.sqrt(b.dot(b))))

    return cs


In [269]:
M_bc.value[100]

u'(944, [1, 5, 7, 49, 59, 64, 70, 92, 94, 95, 102, 109, 198, 222, 244, 246, 268, 275, 276, 279, 286, 291, 299, 307, 311, 339, 363, 387, 394, 405, 406, 407, 417, 429, 430, 435, 437, 450, 456, 472, 479, 483, 495, 497, 508, 537, 545, 620, 622, 653, 660, 677, 715, 727, 749, 751, 757, 774, 805, 807, 822, 823, 826, 843, 846, 865, 868, 882, 886, 890, 896, 916, 932], [-1.511833611467599, 2.185886070341869, 1.140890072697736, 0.37428112139292224, 1.1713771100550523, -1.4867660532102538, -0.3745946043754005, -1.1563983001592852, -1.558778596764261, -2.325567461940413, 1.4394197369288122, -2.3482384888074437, 1.8060867665354454, 1.034402690942319, 1.4492429099233868, -0.8619459138374319, -0.8638550811054344, 0.94038347059927, 0.6291336112919108, -0.17758681254101338, 1.3899082325040317, 0.41489385341906404, -1.3709386640476926, -0.6773311117582632, 0.3084417851356105, -0.890371312950514, -1.9738578111096756, 0.7284152388734584, 0.25725225766126414, -0.761551874591021, -0.43649006295051374, -0.375

In [ ]:
user_234 =users.filter(users.user_id == 234)
user_234.show()


In [ ]:
films_234 = user_234.select(user_234.item_id).rdd.flatMap(lambda x: x).collect()
films_234

In [95]:
sorted(film)[1681]

1682

In [96]:
len(film)

1682

In [ ]:
#функция возвращает словарь с расчитанными  близостями по всем фильмам для переданного фильма
from pyspark.sql.types import MapType, FloatType, IntegerType

def DicCos(f1):
    from pyspark.sql.types import MapType, FloatType, IntegerType
    import math
    from pyspark.mllib.linalg import Vectors, VectorUDT

    d={}
    for f in range(0, len(film_bc.value)):
        v1 = int(f1)
        v2 = int(f)+1
        d[int(f)+1] = calc_cosine_similarity(v1,v2)
    return d



In [71]:
Vectors.parse(M[1000])

SparseVector(944, {22: -1.8096, 45: -0.0365, 57: -1.9477, 75: -1.7519, 82: -1.5423, 145: 1.2134, 174: -2.0243, 181: 0.0171, 279: 1.2816, 374: -1.7631, 393: 1.2136, 397: -2.1353, 533: -1.8352, 758: 1.6875, 796: -1.0438, 880: -0.8745, 923: -2.5201})

In [73]:
M[1000]

u'(944, [22, 45, 57, 75, 82, 145, 174, 181, 279, 374, 393, 397, 533, 758, 796, 880, 923], [-1.809638153192036, -0.03651071691017682, -1.9477296824274184, -1.7518813871930132, -1.5423386905638, 1.2134377070373579, -2.024292442892106, 0.017094792582263096, 1.2816494104585705, -1.763108361397793, 1.2135811367263232, -2.1353216159249553, -1.835150359311199, 1.6875458179014355, -1.0437930444963843, -0.8745169201577601, -2.5201388778297176])'

In [ ]:
#для каждого фильма сформируем запись со словарем близостей до остальных фильмов
from pyspark.mllib.linalg import Vectors, VectorUDT

my_udf = udf(DicCos, MapType(IntegerType(), FloatType()))

films_bu = films_bu.withColumn("S", my_udf("item_id"))


In [446]:
films_bu.show(5)

+-------+--------------------+--------------------+
|item_id|                  bf|                   S|
+-------+--------------------+--------------------+
|    675|0.044759279358367675|Map(645 -> 0.0467...|
|    829|-0.42705134001999223|Map(645 -> 0.0, 8...|
|    467| 0.16690701840272257|Map(645 -> -0.077...|
|   1159|0.006318234718819636|Map(645 -> 0.0591...|
|   1090| -0.5930556738185063|Map(645 -> 0.0081...|
+-------+--------------------+--------------------+
only showing top 5 rows



In [125]:
#просто посмотреть выгрузила
films_bu.select('S').limit(5).rdd.flatMap(lambda x: x).map(lambda x:(x)).collect()

[{1: 0.008555709384381771,
  2: -0.02883322723209858,
  3: -0.04270428419113159,
  4: 0.12829047441482544,
  5: -0.06818822771310806,
  6: -0.06437832862138748,
  7: 0.035668645054101944,
  8: 0.04760006442666054,
  9: -0.005488790571689606,
  10: 0.09038104861974716,
  11: 0.02520112693309784,
  12: 0.07690992951393127,
  13: 0.06596440076828003,
  14: 0.08212143927812576,
  15: -0.06408310681581497,
  16: 0.010476943105459213,
  17: 0.024840442463755608,
  18: 0.10370282828807831,
  19: 0.09394246339797974,
  20: 0.06174915283918381,
  21: 0.054795026779174805,
  22: 0.027565481141209602,
  23: 0.04767393320798874,
  24: 0.006693240720778704,
  25: -0.024223297834396362,
  26: 0.06187634542584419,
  27: -0.06251014769077301,
  28: 0.034633152186870575,
  29: -0.05780913308262825,
  30: -0.02265196479856968,
  31: 0.010924506932497025,
  32: -0.006956496741622686,
  33: 0.0018954473780468106,
  34: 0.00559791037812829,
  35: 0.0,
  36: 0.020404886454343796,
  37: -0.03159795701503754,

In [130]:
#проверяю работу функции просто
what = DicCos(829)

In [131]:
what

{1: -0.04618990544654332,
 2: -0.023265394816316783,
 3: -0.0031355287164808476,
 4: 0.03733757200728171,
 5: -0.026742814169224343,
 6: 0.002301062050169006,
 7: 0.06980755173328033,
 8: -0.0729054615924834,
 9: 0.030612258594722393,
 10: -0.02626700968060842,
 11: -0.004884804588922447,
 12: -0.09542414558524046,
 13: 0.01054163775793412,
 14: 0.019577879658624116,
 15: 0.017391459712604484,
 16: -0.06381455096015197,
 17: 0.04060996547192113,
 18: 0.002299245144342681,
 19: 0.011127281274970192,
 20: 0.019739906294665245,
 21: 0.05462141232300532,
 22: 0.003930458491109268,
 23: -0.05668581684384543,
 24: 0.09685871760224407,
 25: -0.025279135773463107,
 26: 0.00046533441877442163,
 27: -0.005451222088517922,
 28: -0.01776435014686199,
 29: 0.05099675217271895,
 30: 0.0,
 31: 0.05341092729648976,
 32: -0.039071978870221756,
 33: -0.008287662718283805,
 34: 0.15425672623793213,
 35: -0.02513388801350732,
 36: -0.011317148676485084,
 37: -0.04094955151649506,
 38: 0.09226662088207986,

Для каждого фильма, по которому у данного пользователя не стоит рейтинг, найдите:
[a] 30 ближайших фильмов-соседей для этого фильма (среди всех фильмов, а не фильмов, оценённых пользователем).

[b] прогноз оценки пользователя по формуле (базовый предиктор из пункта 4, часть 3). Здесь S(i)- множество фильмов-соседей для фильма i, по которым у данного пользователя есть оценка. 




In [198]:
#выгружаю все фильмы 234 пользователя
user_234 = users_b.filter(users_b.user_id ==234)
films234 = user_234.select(user_234.item_id).rdd.flatMap(lambda x: x).collect()

In [199]:
#забродкастим их
films234_bc = sc.broadcast(films234)

In [447]:
# функция возвращает топ30 id соседей для переданного словаря
def top30(d):
    from collections import Counter
    max_list = []
    c = Counter(d)
    res = c.most_common(31)
    for max_vals in res:
        max_list.append(max_vals[0])
    del max_list[0]
    return max_list

In [ ]:
#вернем 30 соседей
from pyspark.sql.types import MapType, ArrayType

my_top = udf(top30, ArrayType(IntegerType()))

#my_udf = udf(DicCos,FloatType())
films_bu = films_bu.withColumn("item_neb", my_top("S"))


In [467]:
films_bu.show()

+-------+--------------------+--------------------+--------------------+--------------------+
|item_id|                  bf|                   S|            item_neb|        item_neb_234|
+-------+--------------------+--------------------+--------------------+--------------------+
|    829| -0.4270513400199923|Map(645 -> 0.0, 8...|[1209, 1013, 801,...|[477, 925, 566, 808]|
|    296| 0.01581974857595587|Map(645 -> -0.010...|[1630, 1632, 1633...|                [30]|
|    691| 0.05546248651815932|Map(645 -> 0.0, 8...|[1430, 1433, 1432...|     [848, 867, 751]|
|    675|0.044759279358367654|Map(645 -> 0.0467...|[851, 1308, 1307,...|[1456, 656, 671, ...|
|    467| 0.16690701840272257|Map(645 -> -0.077...|[1550, 1554, 814,...|[193, 64, 655, 52...|
|   1090| -0.5930556738185067|Map(645 -> 0.0081...|[1292, 1239, 759,...|                  []|
|   1436|-0.04295878652633444|Map(645 -> 0.0180...|[1634, 1648, 1644...|                  []|
|   1159|0.006318234718819633|Map(645 -> 0.0591...|[1660, 16

In [464]:
#функция отбирает среди отобранных топ-30 соседей фильма просмотренные 234 пользователем
def top_for_234(d):
    new_list =[]
    for item in d:
        if str(item) in films234_bc.value: new_list.append(item)
    return new_list

In [459]:
#на тот случай если дальнейшее расчитаывалось бы для 30 топ среди просмотренных пользователем фильмов, но по замыслу 
#составителей лабы надо было все же из 30 ранее отобранных выбирать просмотренные
def top30_for_234(d):
    from collections import Counter
    max_list = []
    for item in d.keys():
        if str(item) not in films234_bc.value: del d[item]
    c = Counter(d)
    res = c.most_common(31)
    for max_vals in res:
        if max_vals[1]<1: max_list.append(max_vals[0])

    return max_list

In [465]:
#добавим новую колонку с отфильтрованными просмотренными среди топ-30 фильмами
my_top_n = udf(top_for_234, ArrayType(IntegerType()))

films_bu = films_bu.withColumn("item_neb_234", my_top_n("item_neb"))


In [466]:
films_bu.show(30)

+-------+--------------------+--------------------+--------------------+--------------------+
|item_id|                  bf|                   S|            item_neb|        item_neb_234|
+-------+--------------------+--------------------+--------------------+--------------------+
|    467| 0.16690701840272257|Map(645 -> -0.077...|[1550, 1554, 814,...|[193, 64, 655, 52...|
|   1090| -0.5930556738185064|Map(645 -> 0.0081...|[1292, 1239, 759,...|                  []|
|    691| 0.05546248651815934|Map(645 -> 0.0, 8...|[1430, 1433, 1432...|     [848, 867, 751]|
|    675| 0.04475927935836766|Map(645 -> 0.0467...|[851, 1308, 1307,...|[1456, 656, 671, ...|
|    829|-0.42705134001999234|Map(645 -> 0.0, 8...|[1209, 1013, 801,...|[477, 925, 566, 808]|
|   1512| 0.10401325746334374|Map(645 -> -0.026...|[1449, 1558, 1593...|[1449, 557, 617, ...|
|   1159|0.006318234718819621|Map(645 -> 0.0591...|[1660, 1659, 1246...|        [1456, 1003]|
|    296|0.015819748575955864|Map(645 -> -0.010...|[1630, 16

In [468]:
#для дальнейшей оценки нам понадобится отфильтровать фильмы, которые не смотрел 234 пользователь - наилучшие по
#предсказанным оценкам среди них мы и будем рекомендовать
film_filtered = films_bu.filter(~films_bu.item_id.isin(films234)).select('item_id','item_neb_234','bf')

In [ ]:
film_filtered.show()


In [469]:
#далее ожидается джойн с основной таблицей, чтобы не путаться, переменуем в этой столбцы, выделим только нужную нам
#парочку и далее разобьем каждый лист в строке столбца на строчки функцией explode
film_filtered1 = film_filtered.select(col('item_id').alias('item_rec'), col('bf'), \
                                      F.explode(film_filtered.item_neb_234).alias("item_s"))

In [470]:
#посчитаем близость между фильмом для рекомендации и каждым его фильмом-соседом
S_film =udf(lambda A: calc_cosine_similarity(A[0],A[1]), FloatType())

film_filtered1 = film_filtered1.withColumn("S_item_s", S_film(array('item_s','item_rec')))


In [378]:
film_filtered1.show()

+--------+------------------+------+----------+
|item_rec|                bf|item_s|  S_item_s|
+--------+------------------+------+----------+
|     467|0.1669070184027226|   193|0.25755367|
|     467|0.1669070184027226|    64|0.23739986|
|     467|0.1669070184027226|   655|0.23289509|
|     467|0.1669070184027226|   527|0.23261325|
|     467|0.1669070184027226|   317|0.22653647|
|     467|0.1669070184027226|   445|0.22646703|
|     467|0.1669070184027226|   848|0.22410317|
|     467|0.1669070184027226|   186|0.21684764|
|     467|0.1669070184027226|   867|0.21402921|
|     467|0.1669070184027226|   792|0.20693554|
|     467|0.1669070184027226|   843|0.20313245|
|     467|0.1669070184027226|   506|0.20181878|
|     467|0.1669070184027226|   571|0.18047033|
|     467|0.1669070184027226|   735|0.17292777|
|     467|0.1669070184027226|   427|0.17204174|
|     467|0.1669070184027226|   686|0.17096771|
|     467|0.1669070184027226|   152|0.17038855|
|     467|0.1669070184027226|   842|0.17

In [ ]:
film_filtered1.count()

In [471]:
#ну вот начнем расчет, нам понадобятся строки с 234-ым пользователем, чтобы понять, как он оценил фильмы - соседи 
#джоин с основной таблицей
ResTab = film_filtered1.join(users_b, users_b.item_id == film_filtered1.item_s,'inner').\
                        select('item_rec', 'item_s','user_id',col('S_item_s').cast(FloatType()),'bf','r-b')

In [ ]:
ResTab

In [472]:
#фильтруем данные 234 пользователя, нас интересует как он оценил соседей
ResTab= ResTab.filter(ResTab.user_id==234)
ResTab.sort('item_rec').show()

+--------+------+-------+----------+--------------------+--------------------+
|item_rec|item_s|user_id|  S_item_s|                  bf|                 r-b|
+--------+------+-------+----------+--------------------+--------------------+
|    1001|   163|    234|0.16141865| -0.5548444555036163|-0.12364280943486561|
|    1001|   412|    234|  0.155882| -0.5548444555036163|-0.31521975357079146|
|    1001|  1003|    234|0.15115118| -0.5548444555036163| -0.8804768270118695|
|    1001|   571|    234|0.19058542| -0.5548444555036163| -0.6653033650319466|
|    1001|   878|    234|0.16482022| -0.5548444555036163| -0.6140274039862823|
|    1001|   768|    234|0.15310758| -0.5548444555036163| -0.8497041654153779|
|    1002|   163|    234| 0.2067246|-0.32205709910596764|-0.12364280943486561|
|    1002|   236|    234|0.20922074|-0.32205709910596764|-0.19095558610680552|
|    1002|  1003|    234|0.28841257|-0.32205709910596764| -0.8804768270118695|
|    1005|  1448|    234|0.18980479| 0.0913539665907

In [473]:
#начинаем расчитывать результирующий рейтинг-оценку
ResTab1 = ResTab.groupby('item_rec').agg((F.sum(F.abs(col('S_item_s')))).alias('S_abs'),\
                                       F.first('bf').alias('bf'),\
                                       F.sum(col('r-b')*col('S_item_s')).alias('r-b_s')).\
    select('item_rec',(col('bf')+Mr+bu234+(col('r-b_s')/col('S_abs'))).alias('RES_r')).cache()

ResTab1.show()

+--------+------------------+
|item_rec|             RES_r|
+--------+------------------+
|     467|3.1186038940095897|
|    1512|  3.18413042450331|
|     691|3.1199199768755284|
|     829| 1.445923435461828|
|     296|3.7963926524352924|
|    1159| 3.135582002446862|
|     944|2.8691838092054485|
|    1372|3.0243329520533515|
|     926| 2.014761894990765|
|     919|3.3337320180246417|
|    1241|  3.21491818814434|
|    1265|2.3785913563351357|
|    1361| 3.037864190509748|
|     870|2.8274366924270065|
|    1280|2.4784541570816363|
|      51| 2.637806899578879|
|    1500|2.7298513250687995|
|     718| 2.808990081264774|
|     475|2.6807306094621786|
|     574| 3.516680106857957|
+--------+------------------+
only showing top 20 rows



In [474]:
ResTab1.sort('item_rec').show(100)

+--------+------------------+
|item_rec|             RES_r|
+--------+------------------+
|    1001| 2.002889050426915|
|    1002|2.3556274030765474|
|    1005| 3.054187683686964|
|    1006|2.2168927550003783|
|    1007|3.6655045868683276|
|    1008| 2.718223826036709|
|    1009| 2.931776753254996|
|     101|2.7184015651620252|
|    1012| 3.347677177380006|
|    1013| 2.400375244959231|
|    1014| 2.045741001236591|
|    1016|2.5650937377938994|
|    1017|3.0316806833685237|
|    1019|3.6311245995694534|
|    1022| 4.969409112011631|
|    1023|2.1453749044572104|
|    1025|2.5152680291142997|
|    1026|2.9569768519475432|
|    1028| 2.278776607567764|
|    1029|1.9726738421429255|
|     103| 1.639062289432923|
|    1030|1.6887000064532978|
|    1031|3.0827847734143776|
|    1032|2.4442819122272113|
|    1033|2.3244496830615926|
|    1036| 2.674442103696633|
|    1037|2.9934249986025385|
|    1038| 2.485425858376127|
|    1041| 2.380110585160038|
|    1042|2.8675785864953127|
|    1045|

Рекомендуйте пользователю 10 фильмов (predicators_top10) с самыми высокими оценками из пункта 3, часть 4.



In [475]:
#отберем топ-10
ResTab_with_neg = ResTab1.select(col('item_rec').cast(IntegerType()),F.round('Res_r', 4).alias('Res_r')).\
                                 orderBy(desc("Res_r"), col("item_rec"))

In [476]:
predicators_top10 = ResTab_with_neg.select('item_rec').limit(10).\
                        rdd.flatMap(lambda x: x).collect()

In [477]:
predicators_top10

[1022, 1238, 740, 408, 306, 1176, 1194, 1131, 543, 1190]

При подсчете прогноза по формуле из пункта 3, часть 4 отфильтруйте всех соседей с отрицательной близостью.

Рекомендуйте пользователю 10 фильмов (predicators_positive_top10) с самыми высокими оценками из пункта 5, часть 4.



In [478]:

ResTabpos = ResTab.filter(ResTab.S_item_s>0)


In [479]:
ResTabpos.show()

+--------+------+-------+-----------+--------------------+--------------------+
|item_rec|item_s|user_id|   S_item_s|                  bf|                 r-b|
+--------+------+-------+-----------+--------------------+--------------------+
|    1614|   496|    234| 0.17529729|-0.00257293857221...|  0.4382580344159037|
|     633|   496|    234| 0.18922198|  0.2679619675579323|  0.4382580344159037|
|     201|   558|    234| 0.17639814|0.048840456980934324|  0.7518822057488155|
|    1655|   558|    234| 0.28938535|-0.04413189768666...|  0.7518822057488155|
|     773|   558|    234| 0.35785002| -0.1759466175489669|  0.7518822057488155|
|    1513|   558|    234| 0.19649975|-0.12687316756409064|  0.7518822057488155|
|    1150|   558|    234| 0.24701332|-0.06691453636452267|  0.7518822057488155|
|    1440|   558|    234| 0.23778036|-0.06033789697582...|  0.7518822057488155|
|    1049|   558|    234| 0.14402553| -0.4506241070828753|  0.7518822057488155|
|     123|   558|    234| 0.16440444|-0.

In [480]:
ResTabpos.count()

4850

In [ ]:
#лично у меня ничего особо и не отфильтровалась, потому строчки по второму методу будут 1 в1

In [481]:
ResTab.count()

4875

In [482]:
ResTabpos1 = ResTabpos.groupby('item_rec').agg((F.sum(F.abs(col('S_item_s')))).alias('S_abs'),\
                                       F.first('bf').alias('bf'),\
                                       F.sum(col('r-b')*col('S_item_s')).alias('r-b_s')).\
    select('item_rec',(col('bf')+Mr+bu234+(col('r-b_s')/col('S_abs'))).alias('RES_r')).cache()

ResTab_with_pos = ResTabpos1.select(col('item_rec').cast(IntegerType()),F.round('Res_r', 4).alias('Res_r')).\
                                 orderBy(desc("Res_r"), col("item_rec"))

In [483]:
ResTab_with_pos.show()

+--------+------+
|item_rec| Res_r|
+--------+------+
|    1022|4.9694|
|    1238|4.8962|
|     740|4.4101|
|     408|4.2686|
|     306|4.2334|
|    1176|4.0913|
|    1194|4.0777|
|    1131|4.0627|
|     543|4.0455|
|    1190|4.0086|
|     522|3.9856|
|     639|3.9734|
|     903|3.9582|
|     201|3.9319|
|     303|3.9268|
|    1226|3.9261|
|     971|3.9233|
|    1153|3.9171|
|     593|3.9088|
|     592|3.9032|
+--------+------+
only showing top 20 rows



In [484]:
predicators_positive_top10 = ResTab_with_pos.select('item_rec').limit(10).\
                        rdd.flatMap(lambda x: x).collect()

In [485]:
predicators_positive_top10

[1022, 1238, 740, 408, 306, 1176, 1194, 1131, 543, 1190]

In [486]:
import json
open('lab08s.json', 'w').write(
    json.dumps(
        {"average_rating": Mr,
        "predicators_top10": predicators_top10,
        "predicators_positive_top10": predicators_positive_top10}
    )
)